In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error
from sklearn import datasets, linear_model
from sklearn.feature_extraction import DictVectorizer

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC, SVR
from sklearn.model_selection import KFold
import sklearn
import warnings
warnings.filterwarnings('ignore')

import statsmodels.api as sm

In [3]:
# f = pd.read_csv('tg_join_lab.csv', sep='|')
f = pd.read_pickle('tg_join_lab.pkl')
# print(f.shape)
# f['obs'] = 1
# f = f.sort_values('date', ascending=False)

In [4]:
# f = f[f.ticker.isin(['sber', 'gazp', 'lkoh', 'vtbr'])]

In [5]:
data = f.text_reg_stop_lem
target = f.ticker
data.shape, target.shape

((125165,), (125165,))

In [6]:
# !pip install gensim
from gensim.models import Word2Vec

proc_words = [text.split() for text in data]
embeddings_trained = Word2Vec(proc_words, # data for model to train on
                 vector_size=100,                 # embedding vector size
                 min_count=3,             # consider words that occured at least 5 times
                 window=5).wv

In [7]:
def vectorize_sum(comment, embeddings):
    """
    implement a function that converts preprocessed comment to a sum of token vectors
    """
    embedding_dim = embeddings.vectors.shape[1]
    features = np.zeros([embedding_dim], dtype='float32')

    for word in comment.split():
        if word in embeddings:
            features += embeddings[f'{word}']
    
    return features

In [8]:
X_wv = np.stack([vectorize_sum(text, embeddings_trained) for text in data])

In [9]:
X_wv = [list(i) for i in X_wv]

In [10]:
f['emb'] = X_wv

In [11]:
f['bh'] = 1

In [12]:
label_1_0.05
label_1_0.1
label_1_0.2
label_1_0.3
label_1_0.5
label_1_0.8
label_1_1.0
label_1_2.0
label_1_3.0
label_1_4.0
label_1_5.0
label_1_6.0
label_1_7.0
label_1_8.0
label_1_9.0
label_1_10.0

SyntaxError: invalid syntax (Temp/ipykernel_14148/3013829595.py, line 1)

In [13]:
f.ticker.value_counts().tail(30)

bane    491
agro    478
rlmn    458
iskj    458
unac    451
kmaz    445
nmtp    443
flot    442
trmk    388
tanl    383
life    374
gche    369
enru    352
irkt    339
svav    289
sibn    273
lsrg    269
ingr    258
usbn    243
msng    228
etln    219
mstt    214
selg    197
tgkb    182
kzos    157
gltr    154
obuv    151
cntl    150
arsa    128
acko      9
Name: ticker, dtype: int64

In [ ]:
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
field = 'emb'
out = pd.DataFrame()

# min_dfs = [5, 10, 15, 21, 28, 36, 45, 55, 66] 
# max_dfs = [0.5]
# ngram_range_maxs = [1]

# days = [1]

# min_dfs = [5] 
# max_dfs = [0.5]
# ngram_range_maxs = [1]
j_neuts = [0.05]
j_poss = [8.0] 
j_negs = [8.0]
days = [1]





counter = 0
i = 1
t = 4

vector_sizes = [30, 50, 100, 150, 300]
min_counts = [3, 7, 10, 20]
windows = [3, 7, 10, 20]
# vector_sizes = [30]
# min_counts = [3, 7]
# windows = [3]
    
    
    
    

# split
for j_neut in j_neuts:
    for j_pos in j_poss:
        for j_neg in j_negs:
            label_neut = f'label_{i}_{j_neut}'
            label_pos = f'label_{i}_{j_pos}'
            label_neg = f'label_{i}_{j_neg}'
            for day in days:
                for vector_size_i in vector_sizes:
                    for min_count_i in min_counts:
                        for window_i in windows:
                            for ticker in f.ticker.unique(): #['bane']: # 
                                print(ticker)
                                try:
                                    embeddings_trained = Word2Vec(proc_words, # data for model to train on
                                                                  vector_size=vector_size_i,                 # embedding vector size
                                                                  min_count=min_count_i,             # consider words that occured at least 5 times
                                                                  window=window_i).wv

                                    X_wv = np.stack([vectorize_sum(text, embeddings_trained) for text in data])

                                    f['emb'] = [list(i) for i in X_wv]

                                    ps = f[[label_neut, label_pos, label_neg, field]][(f.ticker!=ticker)] 
                                    ps = ps.loc[:,~ps.columns.duplicated()]
                                    if t == 0:
                                        min_df = min(ps[(ps[label_pos]==1)].shape[0], \
                                                     ps[(ps[label_neg]==-1)].shape[0])

                                        ps1 = ps[(ps[label_pos]==1)].sample(frac=1, random_state=41)
                                        ps2 = ps[(ps[label_neg]==-1)].sample(frac=1, random_state=42)

                                        df = pd.concat([ps1[:min_df].rename(columns={label_pos:'sentiment'}), \
                                                        ps2[:min_df].rename(columns={label_neg:'sentiment'}), \
                                                       ]).sample(frac=1, random_state=44)

                                    else:
                                        min_df = min(ps[(ps[label_pos]==1)].shape[0], \
                                                     ps[(ps[label_neut]==0)].shape[0], \
                                                     ps[(ps[label_neg]==-1)].shape[0])

                                        ps1 = ps[(ps[label_pos]==1)][[label_pos, field]].sample(frac=1, random_state=41)
                                        ps2 = ps[(ps[label_neut]==0)][[label_neut, field]].sample(frac=1, random_state=42)
                                        ps3 = ps[(ps[label_neg]==-1)][[label_neg, field]].sample(frac=1, random_state=43)

                                        df = pd.concat([ps1[:min_df].rename(columns={label_pos:'sentiment'}), \
                                                        ps2[:min_df].rename(columns={label_neut:'sentiment'}), \
                                                        ps3[:min_df].rename(columns={label_neg:'sentiment'}) \
                                                       ]).sample(frac=1, random_state=44)


                                    x_train = [list(i) for i in df[field]] 
                                    y_train = df['sentiment'] 

                                # fit
                #                     for ngram_range_max in ngram_range_maxs:
                #                         for min_df in min_dfs:
                #                             for max_df in max_dfs:

                #                                 tf_idf = TfidfVectorizer(min_df=min_df, \
                #                                                          max_df=max_df, \
                #                                                          ngram_range=(1, ngram_range_max))

                                    train_features = x_train


                                    model = DecisionTreeClassifier(min_samples_leaf=2, random_state=7, max_depth=6)
                                    model.fit(train_features, y_train)


                                    y_train_pred = model.predict(train_features)
                # В идеале волатильность взять за эти же лаги наверное
                                    df_temp = pd.DataFrame(f[(f.ticker==ticker)]\
                                                           [[field, 'obs', 'date',\
                                                             f'change_close_{day}', 'bh', 'close']]).\
                reset_index().drop('index', axis=1)

                                    y_pred = model.predict([list(i) for i in df_temp[field]])



                                    df_temp['sentiment'] = y_pred

                                    df_temp['neg'] = 0
                                    df_temp['neg'][df_temp.sentiment==-1] = 1

                                    df_temp['pos'] = 0
                                    df_temp['pos'][df_temp.sentiment==1] = 1

                                    df_temp['neut'] = 0
                                    df_temp['neut'][df_temp.sentiment==0] = 1
                                    df_temp.emb = df_temp.emb.astype(str)
                                    df_temp = df_temp.drop_duplicates().\
                                    groupby(['date', f'change_close_{day}', 'bh', 'close']).sum().reset_index()

                # Делаем сум роллинг окно для суммирования сентиментов для периодов больше 1 дня 
                # Нужно удостовериться в сортировке должно быть по возрастанию даты

                                    df_temp[f'close_lag_1'] = df_temp.close.shift(periods=1)
                                    #df_temp[f'close_change_lag_1'] = df_temp.close.shift(periods=1)
                                    df_temp = df_temp.sort_values('date', ascending=True)

                                    df_temp['neg'] = df_temp['neg'].rolling(day).sum()
                                    df_temp['pos'] = df_temp['pos'].rolling(day).sum()
                                    df_temp['neut'] = df_temp['neut'].rolling(day).sum()

                                    df_temp.dropna(subset=['neg', 'pos', 'neut', 'close_lag_1'], inplace=True)
                    # просто сумма
                                    df_temp['sum_sentiment'] = 0
                                    df_temp['sum_sentiment'][(df_temp.pos>df_temp.neg)&(df_temp.pos>=df_temp.neut)] = 1
                                    df_temp['sum_sentiment'][(df_temp.pos<df_temp.neg)&(df_temp.neg>=df_temp.neut)] = -1

                # дивергенция c добавлением 
                                    df_temp['diver'] = ((df_temp.pos - df_temp.neg)**2 \
                                                              / (df_temp.pos + df_temp.neg)**2)**(1/2)

                                    #df_temp.reset_index(level=[f'change_close_{day}', 'date', 'bh'], inplace=True)
                                    df_temp['clear_return'] = df_temp.sum_sentiment * df_temp[f'change_close_{day}']
                                    df_temp['diver_return'] = df_temp.clear_return * df_temp.diver                

                                    res = dict()
                                    res['counter'] = [counter]
                                    res['ticker'] = [ticker]
                                    res['date'] = [day]

                                    res['min_df'] = [min_df]

                                    res['j_neut'] = [j_neut]
                                    res['j_pos'] = [j_pos] 
                                    res['j_neg'] = [j_neg] 

                                    res['train_size'] = [y_train.shape[0]]
                                    res['test_size'] = [y_pred.shape[0]]

                                    res['obs'] = \
                                    [(df_temp.obs[df_temp.clear_return.isna()==False]).sum()]
                                    res['bh'] = df_temp.bh.max()
                                    res['original_return_isna_all'] = \
                                    [(df_temp[f'change_close_{day}']\
                                      [df_temp[f'change_close_{day}'].isna()==False]).sum()]


                                    res['model_return_isna_all'] = \
                                    [(df_temp.clear_return[df_temp.clear_return.isna()==False]).sum()]

                                    res['model_return_isna_short'] = \
                                    [(df_temp.clear_return[(df_temp.clear_return.isna()==False) & \
                                                                 (df_temp.sum_sentiment!=1)]).sum()]

                                    res['model_return_isna_long'] = \
                                    [(df_temp.clear_return[(df_temp.clear_return.isna()==False) & \
                                                                 (df_temp.sum_sentiment!=-1)]).sum()]


                                    res['model_diver_return_isna_all'] = \
                                    [(df_temp.diver_return[df_temp.diver_return.isna()==False]).sum()]

                                    res['model_diver_return_isna_short'] = \
                                    [(df_temp.diver_return[(df_temp.diver_return.isna()==False) & \
                                                                 (df_temp.sum_sentiment!=1)]).sum()]

                                    res['model_diver_return_isna_long'] = \
                                    [(df_temp.diver_return[(df_temp.diver_return.isna()==False) & \
                                                                 (df_temp.sum_sentiment!=-1)]).sum()]






                                    res['train_accuracy'] = [metrics.accuracy_score(y_train_pred, y_train)]


                                    res['train_precision_macro'] = [metrics.precision_score(y_train_pred, y_train\
                                                                                            , average='macro')]


                                    res['train_precision_micro'] = [metrics.precision_score(y_train_pred, y_train\
                                                                                            , average='micro')]


                                    res['train_recall_micro'] = [metrics.recall_score(y_train_pred, y_train\
                                                                                      , average='micro')]


                                    res['train_recall_macro'] = [metrics.recall_score(y_train_pred, y_train\
                                                                                      , average='macro')]

                                    res['train_f1_micro'] = [metrics.f1_score(y_train_pred, y_train\
                                                                              , average='micro')]


                                    res['train_f1_macro'] = [metrics.f1_score(y_train_pred, y_train\
                                                                              , average='macro')]



                                    x = df_temp[['sum_sentiment', 'close_lag_1']]
                                    x.sum_sentiment = x.sum_sentiment.astype(str)

                                    x = pd.get_dummies(x, drop_first=True)
                                    y = df_temp.close
                                    x2 = sm.add_constant(x)
                                    lm = sm.OLS(y, x2)
                                    res_lm = lm.fit()

                                    res['const'] = res_lm.params[0]
                                    res[f'{x.columns[0]}_coef'] = res_lm.params[1]
                                    res[f'{x.columns[1]}_coef'] = res_lm.params[2]
                                    res[f'{x.columns[2]}_coef'] = res_lm.params[3]

                                    res['const_p'] = res_lm.pvalues[0]
                                    res[f'{x.columns[0]}_p'] = res_lm.pvalues[1]
                                    res[f'{x.columns[1]}_p'] = res_lm.pvalues[2]
                                    res[f'{x.columns[2]}_p'] = res_lm.pvalues[3]
                                    res['r2'] = res_lm.rsquared
                                    res['nobs'] = res_lm.nobs
                                    
                                    res['vector_size'] = vector_size_i
                                    res['min_count'] = min_count_i
                                    res['window'] = window_i
                                    counter += 1
                                    print(counter, end='\r')

                                    
                                    res = pd.DataFrame.from_dict(res)

                                    out = out.append(pd.DataFrame(res))
                                    if counter % 100 == 0: 
            #                             out.to_excel(f'dt_model_ind_final.xlsx', index=False)
                                            out.to_excel(f'dt_model_ind_final_emb.xlsx', index=False)

                                except:
                                    continue
    

                    
out.to_excel(f'dt_model_ind_final_emb.xlsx', index=False)

# out.to_excel(f'dt_model_ind_final.xlsx', index=False)
out

yndx
lsng
aflt
qiwi
rasp
gmkn
five
hydr
rbcm
sngs
pikk
tors
moex
rtkm
tgkd
tcsg
bane
belu
iskj
irkt
irao
lkoh
mgnt
vtbr


In [40]:
# out.to_excel(f'dt_model_ind_final.xlsx', index=False)

In [20]:
out.columns

Index(['counter', 'ticker', 'date', 'min_df', 'j_neut', 'j_pos', 'j_neg',
       'train_size', 'test_size', 'obs', 'bh', 'original_return_isna_all',
       'model_return_isna_all', 'model_return_isna_short',
       'model_return_isna_long', 'model_diver_return_isna_all',
       'model_diver_return_isna_short', 'model_diver_return_isna_long',
       'train_accuracy', 'train_precision_macro', 'train_precision_micro',
       'train_recall_micro', 'train_recall_macro', 'train_f1_micro',
       'train_f1_macro', 'const', 'close_lag_1_coef', 'sum_sentiment_0_coef',
       'sum_sentiment_1_coef', 'const_p', 'close_lag_1_p', 'sum_sentiment_0_p',
       'sum_sentiment_1_p', 'r2', 'nobs'],
      dtype='object')

In [21]:
out.ticker.unique().shape

(76,)

In [24]:
out.ticker[out.sum_sentiment_0_p<=0.05].unique().shape

(72,)

In [25]:
out.ticker[out.original_return_isna_all<=out.model_return_isna_all].unique().shape

(62,)

In [37]:

out.ticker[out.train_f1_macro>=0.55].unique().shape

(75,)

In [43]:
out.groupby('ticker')['train_f1_macro'].mean().sort_values()

ticker
mtss    0.472128
five    0.473109
msng    0.473399
mvid    0.474249
yndx    0.474389
          ...   
gche    0.480945
vtbr    0.482246
gazp    0.483338
rual    0.483514
tcsg    0.486835
Name: train_f1_macro, Length: 76, dtype: float64

In [50]:
out.groupby('j_neut')['train_f1_macro'].mean()

j_neut
0.05    0.482275
0.10    0.481878
0.20    0.473106
0.50    0.473464
Name: train_f1_macro, dtype: float64

In [47]:
out.groupby('j_pos')['train_recall_macro'].mean()

j_pos
2.0    0.592040
3.0    0.587449
4.0    0.584196
6.0    0.573573
8.0    0.582956
Name: train_recall_macro, dtype: float64

In [49]:
out.groupby('j_neg')['train_f1_macro'].mean()

j_neg
2.0    0.440971
3.0    0.451597
4.0    0.464516
6.0    0.494132
8.0    0.537229
Name: train_f1_macro, dtype: float64

In [4]:
j_neut = 0.2
j_pos = 2.0
j_neg = 2.0
i = 1
label_neut = f'label_{i}_{j_neut}'
label_pos = f'label_{i}_{j_pos}'
label_neg = f'label_{i}_{j_neg}'
print('neut: ', f[label_neut][f[label_neut]==0].shape)
print('neg: ', f[label_neg][f[label_neg]==-1].shape)
print('pos: ', f[label_pos][f[label_pos]==1].shape)

neut:  (15389,)
neg:  (14162,)
pos:  (16278,)


In [8]:
f['bh'] = 1

In [11]:
from sklearn import metrics
field = 'text_reg_stop_lem'
out = pd.DataFrame()

# min_dfs = [5, 10, 15, 21, 28, 36, 45, 55, 66] 
# max_dfs = [0.5]
# ngram_range_maxs = [1]
# j_neuts = [0.2, 0.5]
# j_poss = [2.0, 3.0, 4.0, 6.0] 
# j_negs = [2.0, 3.0, 4.0, 6.0]
# days = [1]

min_dfs = [5] 
max_dfs = [0.5]
ngram_range_maxs = [1]
j_neuts = [0.2, 0.5]
j_poss = [2.0] 
j_negs = [2.0]
days = [1]


counter = 0
i = 1
t = 4

# split
for ticker in  ['sber', 'vtbr']: #f.ticker.unique(): #
    print(ticker)

    for j_neut in j_neuts:
        for j_pos in j_poss:
            for j_neg in j_negs:
                label_neut = f'label_{i}_{j_neut}'
                label_pos = f'label_{i}_{j_pos}'
                label_neg = f'label_{i}_{j_neg}'
                for day in days:
#                     try:

                    ps = f[[label_neut, label_pos, label_neg, field]][(f.ticker!=ticker)] 
                    ps = ps.loc[:,~ps.columns.duplicated()]
                    if t == 0:
                        min_df = min(ps[(ps[label_pos]==1)].shape[0], \
                                     ps[(ps[label_neg]==-1)].shape[0])

                        ps1 = ps[(ps[label_pos]==1)].sample(frac=1, random_state=41)
                        ps2 = ps[(ps[label_neg]==-1)].sample(frac=1, random_state=42)

                        df = pd.concat([ps1[:min_df].rename(columns={label_pos:'sentiment'}), \
                                        ps2[:min_df].rename(columns={label_neg:'sentiment'}), \
                                       ]).sample(frac=1, random_state=44)

                    else:
                        min_df = min(ps[(ps[label_pos]==1)].shape[0], \
                                     ps[(ps[label_neut]==0)].shape[0], \
                                     ps[(ps[label_neg]==-1)].shape[0])

                        ps1 = ps[(ps[label_pos]==1)][[label_pos, field]].sample(frac=1, random_state=41)
                        ps2 = ps[(ps[label_neut]==0)][[label_neut, field]].sample(frac=1, random_state=42)
                        ps3 = ps[(ps[label_neg]==-1)][[label_neg, field]].sample(frac=1, random_state=43)

                        df = pd.concat([ps1[:min_df].rename(columns={label_pos:'sentiment'}), \
                                        ps2[:min_df].rename(columns={label_neut:'sentiment'}), \
                                        ps3[:min_df].rename(columns={label_neg:'sentiment'}) \
                                       ]).sample(frac=1, random_state=44)


                    x_train = df[field]
                    y_train = df['sentiment'] 

                # fit
                    for ngram_range_max in ngram_range_maxs:
                        for min_df in min_dfs:
                            for max_df in max_dfs:

                                tf_idf = TfidfVectorizer(min_df=min_df, \
                                                         max_df=max_df, \
                                                         ngram_range=(1, ngram_range_max))

                                train_features = tf_idf.fit_transform(x_train)


                                model = MultinomialNB(alpha=0.1)
                                model.fit(train_features, y_train)
                                y_train_pred = model.predict(train_features)
# В идеале волатильность взять за эти же лаги наверное
                                df_temp = pd.DataFrame(f[(f.ticker==ticker)]\
                                                       [[field, 'obs', 'date',\
                                                         f'change_close_{day}', 'bh', 'close']]).\
    reset_index().drop('index', axis=1)

                                y_pred = model.predict(tf_idf.transform(df_temp[field]))

                                df_temp['sentiment'] = y_pred

                                df_temp['neg'] = 0
                                df_temp['neg'][df_temp.sentiment==-1] = 1

                                df_temp['pos'] = 0
                                df_temp['pos'][df_temp.sentiment==1] = 1

                                df_temp['neut'] = 0
                                df_temp['neut'][df_temp.sentiment==0] = 1

                                df_temp = df_temp.drop_duplicates().\
                                groupby(['date', f'change_close_{day}', 'bh', 'close']).sum().reset_index()

# Делаем сум роллинг окно для суммирования сентиментов для периодов больше 1 дня 
# Нужно удостовериться в сортировке должно быть по возрастанию даты

                                df_temp[f'close_lag_1'] = df_temp.close.shift(periods=1)
                                #df_temp[f'close_change_lag_1'] = df_temp.close.shift(periods=1)
                                df_temp = df_temp.sort_values('date', ascending=True)

                                df_temp['neg'] = df_temp['neg'].rolling(day).sum()
                                df_temp['pos'] = df_temp['pos'].rolling(day).sum()
                                df_temp['neut'] = df_temp['neut'].rolling(day).sum()

                                df_temp.dropna(subset=['neg', 'pos', 'neut', 'close_lag_1'], inplace=True)
                # просто сумма
                                df_temp['sum_sentiment'] = 0
                                df_temp['sum_sentiment'][(df_temp.pos>df_temp.neg)&(df_temp.pos>=df_temp.neut)] = 1
                                df_temp['sum_sentiment'][(df_temp.pos<df_temp.neg)&(df_temp.neg>=df_temp.neut)] = -1

# дивергенция c добавлением 
                                df_temp['diver'] = ((df_temp.pos - df_temp.neg)**2 \
                                                          / (df_temp.pos + df_temp.neg)**2)**(1/2)

                                #df_temp.reset_index(level=[f'change_close_{day}', 'date', 'bh'], inplace=True)
                                df_temp['clear_return'] = df_temp.sum_sentiment * df_temp[f'change_close_{day}']
                                df_temp['diver_return'] = df_temp.clear_return * df_temp.diver                

                                res = dict()
                                res['counter'] = [counter]
                                res['ticker'] = [ticker]
                                res['date'] = [day]

                                res['min_df'] = [min_df]

                                res['j_neut'] = [j_neut]
                                res['j_pos'] = [j_pos] 
                                res['j_neg'] = [j_neg] 

                                res['train_size'] = [y_train.shape[0]]
                                res['test_size'] = [y_pred.shape[0]]

                                res['obs'] = \
                                [(df_temp.obs[df_temp.clear_return.isna()==False]).sum()]
                                res['bh'] = df_temp.bh.max()
                                res['original_return_isna_all'] = \
                                [(df_temp[f'change_close_{day}']\
                                  [df_temp[f'change_close_{day}'].isna()==False]).sum()]


                                res['model_return_isna_all'] = \
                                [(df_temp.clear_return[df_temp.clear_return.isna()==False]).sum()]

                                res['model_return_isna_short'] = \
                                [(df_temp.clear_return[(df_temp.clear_return.isna()==False) & \
                                                             (df_temp.sum_sentiment!=1)]).sum()]

                                res['model_return_isna_long'] = \
                                [(df_temp.clear_return[(df_temp.clear_return.isna()==False) & \
                                                             (df_temp.sum_sentiment!=-1)]).sum()]


                                res['model_diver_return_isna_all'] = \
                                [(df_temp.diver_return[df_temp.diver_return.isna()==False]).sum()]

                                res['model_diver_return_isna_short'] = \
                                [(df_temp.diver_return[(df_temp.diver_return.isna()==False) & \
                                                             (df_temp.sum_sentiment!=1)]).sum()]

                                res['model_diver_return_isna_long'] = \
                                [(df_temp.diver_return[(df_temp.diver_return.isna()==False) & \
                                                             (df_temp.sum_sentiment!=-1)]).sum()]






                                res['train_accuracy'] = [metrics.accuracy_score(y_train_pred, y_train)]


                                res['train_precision_macro'] = [metrics.precision_score(y_train_pred, y_train\
                                                                                        , average='macro')]


                                res['train_precision_micro'] = [metrics.precision_score(y_train_pred, y_train\
                                                                                        , average='micro')]


                                res['train_recall_micro'] = [metrics.recall_score(y_train_pred, y_train\
                                                                                  , average='micro')]


                                res['train_recall_macro'] = [metrics.recall_score(y_train_pred, y_train\
                                                                                  , average='macro')]

                                res['train_f1_micro'] = [metrics.f1_score(y_train_pred, y_train\
                                                                          , average='micro')]


                                res['train_f1_macro'] = [metrics.f1_score(y_train_pred, y_train\
                                                                          , average='macro')]



                                x = df_temp[['sum_sentiment', 'close_lag_1']]
                                x.sum_sentiment = x.sum_sentiment.astype(str)

                                x = pd.get_dummies(x, drop_first=True)
                                y = df_temp.close
                                x2 = sm.add_constant(x)
                                lm = sm.OLS(y, x2)
                                res_lm = lm.fit()

                                res['const'] = res_lm.params[0]
                                res[f'{x.columns[0]}_coef'] = res_lm.params[1]
                                res[f'{x.columns[1]}_coef'] = res_lm.params[2]
                                res[f'{x.columns[2]}_coef'] = res_lm.params[3]

                                res['const_p'] = res_lm.pvalues[0]
                                res[f'{x.columns[0]}_p'] = res_lm.pvalues[1]
                                res[f'{x.columns[1]}_p'] = res_lm.pvalues[2]
                                res[f'{x.columns[2]}_p'] = res_lm.pvalues[3]
                                res['r2'] = res_lm.rsquared
                                res['nobs'] = res_lm.nobs
                                counter += 1
                                print(counter, end='\r')


                                res = pd.DataFrame.from_dict(res)

                                out = out.append(pd.DataFrame(res))
#                                     if counter % 100 == 0: 
#                                         out.to_excel(f'/is/res/nb/nb_model_ind_final.xlsx', index=False)
                                    
#                     except:
#                         continue
    

                    


out.to_excel(f'nb_model_ind_final.xlsx', index=False)
out

sber
vtbr


,counter,ticker,date,min_df,j_neut,j_pos,j_neg,train_size,test_size,obs,...,const,close_lag_1_coef,sum_sentiment_0_coef,sum_sentiment_1_coef,const_p,close_lag_1_p,sum_sentiment_0_p,sum_sentiment_1_p,r2,nobs
0,0,sber,1,5,0.2,2.0,2.0,36609,18362,18312,...,1.030217,0.994081,0.646175,0.743559,0.037722,0.0,0.006479,0.005079,0.993880,1542.0
0,1,sber,1,5,0.5,2.0,2.0,36609,18362,18312,...,0.867078,0.994126,0.791563,1.038908,0.081581,0.0,0.000817,0.000185,0.993905,1542.0
0,2,vtbr,1,5,0.2,2.0,2.0,39546,12557,12528,...,0.000460,0.989995,-0.000049,-0.000068,0.000129,0.0,0.349229,0.196240,0.990690,1388.0
0,3,vtbr,1,5,0.5,2.0,2.0,39546,12557,12528,...,0.000409,0.989598,0.000070,-0.000009,0.000600,0.0,0.175544,0.857650,0.990701,1388.0


In [10]:
!dir

 ’®¬ ў гбва®©бвўҐ C Ё¬ҐҐв ¬ҐвЄг Windows-SSD
 ‘ҐаЁ©­л© ­®¬Ґа в®¬ : 6252-A73E

 ‘®¤Ґа¦Ё¬®Ґ Ї ЇЄЁ C:\Users\79627\Desktop\Љђ\¤ ­­лҐ

13.03.2022  14:50    <DIR>          .
13.03.2022  14:50    <DIR>          ..
12.03.2022  22:42    <DIR>          .ipynb_checkpoints
05.03.2022  19:33           116я258 03_text_processing.ipynb
13.03.2022  14:50            46я858 05_tg_NB.ipynb
05.12.2021  22:14           473я071 data.xlsx
21.02.2022  21:40        17я140я695 eikon_all.xlsb
22.02.2022  11:14       465я152я115 read_tg_msg.csv
06.02.2022  22:01            36я402 telegram.xlsx
06.02.2022  19:50        99я247я225 telegram_all.xlsx
05.03.2022  13:05       145я135я260 text_lower_ticker_flag_dt.csv
05.03.2022  13:10       145я135я255 text_lower_tic_all_flag_all_dt_e.csv
05.03.2022  13:10       206я290я098 text_lower_tic_all_flag_all_dt_e_reg.csv
12.03.2022  22:47       425я701я302 tg_join_lab.csv
03.02.2022  15:37       467я303я098 tg_msg.csv
23.02.2022  13:34           148я602 tg_msg.ipynb
12.03.2022

In [ ]:
from sklearn import metrics
field = 'text_reg_spell_stop_lem'
out = pd.DataFrame()

min_dfs = [5, 7, 10, 15, 21, 28, 36, 45, 55, 66] 
max_dfs = [0.5]
ngram_range_maxs = [1]
j_neuts = [0.1, 0.2, 0.5, 0.05]
j_poss = [2.0, 3.0, 5.0, 9.0] 
j_negs = [2.0, 3.0, 5.0, 9.0]
days = [1]


counter = 0
i = 1
t = 4

# split
for ticker in  f.ticker.unique(): #['sber']: #
    print(ticker)

    for j_neut in j_neuts:
        for j_pos in j_poss:
            for j_neg in j_negs:
                label_neut = f'label_{i}_{j_neut}'
                label_pos = f'label_{i}_{j_pos}'
                label_neg = f'label_{i}_{j_neg}'
                for day in days:
                    try:

                        ps = f[[label_neut, label_pos, label_neg, field]][(f.ticker!=ticker)] 
                        ps = ps.loc[:,~ps.columns.duplicated()]
                        if t == 0:
                            min_df = min(ps[(ps[label_pos]==1)].shape[0], \
                                         ps[(ps[label_neg]==-1)].shape[0])

                            ps1 = ps[(ps[label_pos]==1)].sample(frac=1, random_state=41)
                            ps2 = ps[(ps[label_neg]==-1)].sample(frac=1, random_state=42)

                            df = pd.concat([ps1[:min_df].rename(columns={label_pos:'sentiment'}), \
                                            ps2[:min_df].rename(columns={label_neg:'sentiment'}), \
                                           ]).sample(frac=1, random_state=44)

                        else:
                            min_df = min(ps[(ps[label_pos]==1)].shape[0], \
                                         ps[(ps[label_neut]==0)].shape[0], \
                                         ps[(ps[label_neg]==-1)].shape[0])

                            ps1 = ps[(ps[label_pos]==1)][[label_pos, field]].sample(frac=1, random_state=41)
                            ps2 = ps[(ps[label_neut]==0)][[label_neut, field]].sample(frac=1, random_state=42)
                            ps3 = ps[(ps[label_neg]==-1)][[label_neg, field]].sample(frac=1, random_state=43)

                            df = pd.concat([ps1[:min_df].rename(columns={label_pos:'sentiment'}), \
                                            ps2[:min_df].rename(columns={label_neut:'sentiment'}), \
                                            ps3[:min_df].rename(columns={label_neg:'sentiment'}) \
                                           ]).sample(frac=1, random_state=44)


                        x_train = df[field]
                        y_train = df['sentiment'] 

                    # fit
                        for ngram_range_max in ngram_range_maxs:
                            for min_df in min_dfs:
                                for max_df in max_dfs:

                                    tf_idf = TfidfVectorizer(min_df=min_df, \
                                                             max_df=max_df, \
                                                             ngram_range=(1, ngram_range_max))

                                    train_features = tf_idf.fit_transform(x_train)


                                    model = MultinomialNB(alpha=0.1)
                                    model.fit(train_features, y_train)
                                    y_train_pred = model.predict(train_features)
    # В идеале волатильность взять за эти же лаги наверное
                                    df_temp = pd.DataFrame(f[(f.ticker==ticker)]\
                                                           [['text_reg_spell_stop_lem', 'obs', 'date',\
                                                             f'change_close_{day}', 'bh', 'close']]).\
        reset_index().drop('index', axis=1)

                                    y_pred = model.predict(tf_idf.transform(df_temp.text_reg_spell_stop_lem))

                                    df_temp['sentiment'] = y_pred

                                    df_temp['neg'] = 0
                                    df_temp['neg'][df_temp.sentiment==-1] = 1

                                    df_temp['pos'] = 0
                                    df_temp['pos'][df_temp.sentiment==1] = 1

                                    df_temp['neut'] = 0
                                    df_temp['neut'][df_temp.sentiment==0] = 1

                                    df_temp = df_temp.drop_duplicates().\
                                    groupby(['date', f'change_close_{day}', 'bh', 'close']).sum().reset_index()

    # Делаем сум роллинг окно для суммирования сентиментов для периодов больше 1 дня 
    # Нужно удостовериться в сортировке должно быть по возрастанию даты

                                    df_temp[f'close_lag_1'] = df_temp.close.shift(periods=1)
                                    #df_temp[f'close_change_lag_1'] = df_temp.close.shift(periods=1)
                                    df_temp = df_temp.sort_values('date', ascending=True)

                                    df_temp['neg'] = df_temp['neg'].rolling(day).sum()
                                    df_temp['pos'] = df_temp['pos'].rolling(day).sum()
                                    df_temp['neut'] = df_temp['neut'].rolling(day).sum()

                                    df_temp.dropna(subset=['neg', 'pos', 'neut', 'close_lag_1'], inplace=True)
                    # просто сумма
                                    df_temp['sum_sentiment'] = 0
                                    df_temp['sum_sentiment'][(df_temp.pos>df_temp.neg)&(df_temp.pos>=df_temp.neut)] = 1
                                    df_temp['sum_sentiment'][(df_temp.pos<df_temp.neg)&(df_temp.neg>=df_temp.neut)] = -1

    # дивергенция c добавлением 
                                    df_temp['diver'] = ((df_temp.pos - df_temp.neg)**2 \
                                                              / (df_temp.pos + df_temp.neg)**2)**(1/2)

                                    #df_temp.reset_index(level=[f'change_close_{day}', 'date', 'bh'], inplace=True)
                                    df_temp['clear_return'] = df_temp.sum_sentiment * df_temp[f'change_close_{day}']
                                    df_temp['diver_return'] = df_temp.clear_return * df_temp.diver                

                                    res = dict()
                                    res['counter'] = [counter]
                                    res['ticker'] = [ticker]
                                    res['date'] = [day]

                                    res['min_df'] = [min_df]

                                    res['j_neut'] = [j_neut]
                                    res['j_pos'] = [j_pos] 
                                    res['j_neg'] = [j_neg] 

                                    res['train_size'] = [y_train.shape[0]]
                                    res['test_size'] = [y_pred.shape[0]]

                                    res['obs'] = \
                                    [(df_temp.obs[df_temp.clear_return.isna()==False]).sum()]
                                    res['bh'] = df_temp.bh.max()
                                    res['original_return_isna_all'] = \
                                    [(df_temp[f'change_close_{day}']\
                                      [df_temp[f'change_close_{day}'].isna()==False]).sum()]


                                    res['model_return_isna_all'] = \
                                    [(df_temp.clear_return[df_temp.clear_return.isna()==False]).sum()]

                                    res['model_return_isna_short'] = \
                                    [(df_temp.clear_return[(df_temp.clear_return.isna()==False) & \
                                                                 (df_temp.sum_sentiment!=1)]).sum()]

                                    res['model_return_isna_long'] = \
                                    [(df_temp.clear_return[(df_temp.clear_return.isna()==False) & \
                                                                 (df_temp.sum_sentiment!=-1)]).sum()]


                                    res['model_diver_return_isna_all'] = \
                                    [(df_temp.diver_return[df_temp.diver_return.isna()==False]).sum()]

                                    res['model_diver_return_isna_short'] = \
                                    [(df_temp.diver_return[(df_temp.diver_return.isna()==False) & \
                                                                 (df_temp.sum_sentiment!=1)]).sum()]

                                    res['model_diver_return_isna_long'] = \
                                    [(df_temp.diver_return[(df_temp.diver_return.isna()==False) & \
                                                                 (df_temp.sum_sentiment!=-1)]).sum()]






                                    res['train_accuracy'] = [metrics.accuracy_score(y_train_pred, y_train)]


                                    res['train_precision_macro'] = [metrics.precision_score(y_train_pred, y_train\
                                                                                            , average='macro')]


                                    res['train_precision_micro'] = [metrics.precision_score(y_train_pred, y_train\
                                                                                            , average='micro')]


                                    res['train_recall_micro'] = [metrics.recall_score(y_train_pred, y_train\
                                                                                      , average='micro')]


                                    res['train_recall_macro'] = [metrics.recall_score(y_train_pred, y_train\
                                                                                      , average='macro')]

                                    res['train_f1_micro'] = [metrics.f1_score(y_train_pred, y_train\
                                                                              , average='micro')]


                                    res['train_f1_macro'] = [metrics.f1_score(y_train_pred, y_train\
                                                                              , average='macro')]



                                    x = df_temp[['sum_sentiment', 'close_lag_1']]
                                    x.sum_sentiment = x.sum_sentiment.astype(str)

                                    x = pd.get_dummies(x, drop_first=True)
                                    y = df_temp.close
                                    x2 = sm.add_constant(x)
                                    lm = sm.OLS(y, x2)
                                    res_lm = lm.fit()

                                    res['const'] = res_lm.params[0]
                                    res[f'{x.columns[0]}_coef'] = res_lm.params[1]
                                    res[f'{x.columns[1]}_coef'] = res_lm.params[2]
                                    res[f'{x.columns[2]}_coef'] = res_lm.params[3]

                                    res['const_p'] = res_lm.pvalues[0]
                                    res[f'{x.columns[0]}_p'] = res_lm.pvalues[1]
                                    res[f'{x.columns[1]}_p'] = res_lm.pvalues[2]
                                    res[f'{x.columns[2]}_p'] = res_lm.pvalues[3]
                                    res['r2'] = res_lm.rsquared
                                    res['nobs'] = res_lm.nobs
                                    counter += 1
                                    print(counter, end='\r')


                                    res = pd.DataFrame.from_dict(res)
                                    
                                    out = out.append(pd.DataFrame(res))
                                    if counter % 1000 == 0: 
                                        out.to_excel(f'/is/res/nb/nb_model_ind_final2.xlsx', index=False)
                                    
                    except:
                        continue
    

                    


out.to_excel(f'/is/res/nb/nb_model_ind_final2.xlsx', index=False)
out